<a href="https://colab.research.google.com/github/AmericaNC/Alumno/blob/main/intento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install firebase-admin pandas

In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore

cred = credentials.Certificate("serviceAccountKey.json")
firebase_admin.initialize_app(cred)

db = firestore.client()

In [ ]:
def get_users():
    users_ref = db.collection("Usuarios")
    users = {}

    for doc in users_ref.stream():
        users[doc.id] = doc.to_dict()

    return users

In [13]:
def get_seguimiento(user_id):
    seguimiento_ref = (
        db.collection("Usuarios")
        .document(user_id)
        .collection("Seguimiento")
    )

    eventos = []

    print(f"Fetching 'Seguimiento' documents for user: {user_id}")
    seguimiento_docs = list(seguimiento_ref.stream())
    if not seguimiento_docs:
        print(f"No 'Seguimiento' documents found for user {user_id}")

    for fecha_doc in seguimiento_docs:
        fecha = fecha_doc.id
        print(f"Found 'Seguimiento' document for date: {fecha}")

        # Assuming there is a subcollection named "Eventos" under each date document
        eventos_subcollection_ref = fecha_doc.reference.collection("Eventos")
        eventos_sub_docs = list(eventos_subcollection_ref.stream())
        if not eventos_sub_docs:
            print(f"No 'Eventos' subcollection documents found for date {fecha} under user {user_id}")

        for ev in eventos_sub_docs:
            data = ev.to_dict()
            data["fecha"] = fecha
            eventos.append(data)

    return eventos

In [ ]:
def get_tomas(user_id):
    tomas_ref = (
        db.collection("Usuarios")
        .document(user_id)
        .collection("TomasDiarias")
    )

    tomas = []

    for doc in tomas_ref.stream():
        data = doc.to_dict()
        data["fecha"] = doc.id
        tomas.append(data)

    return tomas


In [ ]:
EMOCIONES = {
    "Contento": 1.0,
    "Tranquilo": 0.8,
    "Ansioso": 0.3,
    "Avergonzado": 0.2,
    "Triste": 0.1
}

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

def build_features(user_id, eventos, tomas, ventana_dias=7):
    rows = []

    eventos_df = pd.DataFrame(eventos)
    tomas_df = pd.DataFrame(tomas)

    if eventos_df.empty:
        return rows

    eventos_df["fecha"] = pd.to_datetime(eventos_df["fecha"], errors="coerce")

    start = eventos_df["fecha"].min()
    end = eventos_df["fecha"].max()

    current = start

    while current <= end:
        window_end = current + timedelta(days=ventana_dias)

        window = eventos_df[
            (eventos_df["fecha"] >= current) &
            (eventos_df["fecha"] < window_end)
        ]

        emociones = window[
            window["categoria"] == "Estigma"
        ]["respuesta"].map(EMOCIONES).dropna()

        avg_emocion = emociones.mean() if not emociones.empty else 0.5
        delta_emocion = emociones.iloc[-1] - emociones.iloc[0] if len(emociones) > 1 else 0

        respuestas = len(window)

        dias_inactivo = ventana_dias - window["fecha"].dt.date.nunique()

        # Abandono (definición simple)
        abandono = 1 if respuestas == 0 else 0

        rows.append({
            "user_id": user_id,
            "periodo_inicio": current,
            "avg_emocion": avg_emocion,
            "delta_emocion": delta_emocion,
            "respuestas": respuestas,
            "dias_inactivo": dias_inactivo,
            "abandono": abandono
        })

        current = window_end

    return rows

In [14]:
dataset = []

users = get_users()

for user_id in users.keys():
    eventos = get_seguimiento(user_id)
    tomas = get_tomas(user_id)

    rows = build_features(user_id, eventos, tomas)
    dataset.extend(rows)

df = pd.DataFrame(dataset)
df

Fetching 'Seguimiento' documents for user: 04t0y9pqJVSgQDYt0sWpVbzNkI53
Found 'Seguimiento' document for date: 2025-10-28
No 'Eventos' subcollection documents found for date 2025-10-28 under user 04t0y9pqJVSgQDYt0sWpVbzNkI53
Found 'Seguimiento' document for date: 2025-10-29
No 'Eventos' subcollection documents found for date 2025-10-29 under user 04t0y9pqJVSgQDYt0sWpVbzNkI53
Found 'Seguimiento' document for date: 2025-10-30
No 'Eventos' subcollection documents found for date 2025-10-30 under user 04t0y9pqJVSgQDYt0sWpVbzNkI53
Found 'Seguimiento' document for date: 2025-11-01
No 'Eventos' subcollection documents found for date 2025-11-01 under user 04t0y9pqJVSgQDYt0sWpVbzNkI53
Found 'Seguimiento' document for date: 2025-11-02
No 'Eventos' subcollection documents found for date 2025-11-02 under user 04t0y9pqJVSgQDYt0sWpVbzNkI53
Found 'Seguimiento' document for date: 2025-11-03
No 'Eventos' subcollection documents found for date 2025-11-03 under user 04t0y9pqJVSgQDYt0sWpVbzNkI53
Found 'Seg

""


In [12]:
sample_user_id = list(users.keys())[0]
print(f"Checking 'eventos' for user ID: {sample_user_id}")
sample_eventos = get_seguimiento(sample_user_id)
print(f"Eventos retrieved: {sample_eventos}")

if not sample_eventos:
    print("No events found for this user. This might be the reason why the DataFrame is empty.")
else:
    print("Events found for this user. The issue might be elsewhere.")

Checking 'eventos' for user ID: 04t0y9pqJVSgQDYt0sWpVbzNkI53
Eventos retrieved: []
No events found for this user. This might be the reason why the DataFrame is empty.
